In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pickle

from storage import fit_storage, create_storage_harmonic
from release import fit_release, create_release_harmonic, create_release_linear
from inputs import read_reservoir_attributes, read_reservoir_data#, rank_and_filter_data
from functions import plot_nor, plot_release, epiweek_to_date

from lisfloodreservoirs.utils.metrics import KGEmod

## Configuration 

In [2]:
# list of reservoirs to be trained
reservoirs = pd.read_csv('Z:/nahaUsers/casadje/datasets/reservoirs/ResOpsUS/v1.1/selection/reservoirs.txt',
                         header=None).squeeze().tolist()

In [3]:
USRDATS_path = Path('Z:/nahaUsers/casadje/datasets/reservoirs/ResOpsUS/raw')
GRanD_path = Path('Z:/nahaUsers/casadje/datasets/reservoirs/GRanD/v1_3')

# dam_id = 753
cutoff_year = 1982

In [4]:
path_results = Path('Z:/nahaUsers/casadje/datasets/reservoirs/ResOpsUS/results/starfit')
path_nor = path_results / 'NOR'
path_nor.mkdir(parents=True, exist_ok=True)

path_release = path_results / 'release'
path_release.mkdir(parents=True, exist_ok=True)

## Data

In [ ]:
# # read reservoir attributes and extract storage capacity
# attributes = read_reservoir_attributes(GRanD_path, dam_id)
# Vtot = attributes.loc[dam_id, 'CAP_MCM']

In [ ]:
# # read daily time series
# daily = (
#     read_reservoir_data(USRDATS_path, dam_id)
#     .assign(
#         i=lambda x: x['i_cumecs'] * 1e-6 * 86400,  # MCM/day
#         r=lambda x: x['r_cumecs'] * 1e-6 * 86400,  # MCM/day
#         year=lambda x: x.date.dt.year,
#         epiweek=lambda x: x.date.dt.isocalendar().week
#     )
#     .rename(columns={'s_MCM': 's'})
#     .loc[:, ['date', 's', 'i', 'r', 'year', 'epiweek']]
#     .query('year >= @cutoff_year')
#     .set_index('date')
#     )
# daily.epiweek = daily.epiweek.astype(int)

## Fit reservoir

### Storage functions

In [5]:
for dam_id in tqdm(reservoirs):
    
    # read reservoir attributes and extract storage capacity
    attributes = read_reservoir_attributes(GRanD_path, dam_id)
    Vtot = attributes.loc[dam_id, 'CAP_MCM']

    # fit storage model
    model_storage = fit_storage(dam_id, USRDATS_path, attributes, cutoff_year=cutoff_year)
    
    # export parameters of the storage harmonics
    pd.DataFrame({
                'flood': model_storage["NSR upper bound"],
                'conservation': model_storage["NSR lower bound"]
            }).to_csv(path_nor / f'{dam_id}.csv', index=False)
    
    # define normal operating range (NOR)
    NORup = create_storage_harmonic(model_storage['NSR upper bound'], name='flood').set_index('epiweek')
    NORdown = create_storage_harmonic(model_storage['NSR lower bound'], name='conservation').set_index('epiweek')
    NOR = pd.concat((NORup, NORdown), axis=1)

    # weekly time series of standardised storage combined with NOR
    weekly_storage = (model_storage['weekly storage']
          .merge(NORup, on='epiweek')
          .merge(NORdown, on='epiweek')
          )
    weekly_storage.sort_values(['year', 'epiweek'], inplace=True)
    weekly_storage['date'] = weekly_storage.apply(lambda row: epiweek_to_date(row['year'], row['epiweek']), axis=1)
    weekly_storage.set_index('date', inplace=True)

    # plot
    plot_nor(weekly_storage,
             NOR,
             title='{0} - {1}'.format(*attributes.loc[dam_id, ['GRAND_ID', 'DAM_NAME']]),
             save=path_nor / f'{dam_id}.jpg')

    # # plot time series
    # fig, ax = plt.subplots(figsize=(12, 4))
    # ax.plot(weekly_storage.s_pct, c='k', lw=1)
    # ax.fill_between(weekly_storage.index, weekly_storage.flood, 1, color='whitesmoke', zorder=0)
    # ax.fill_between(weekly_storage.index, weekly_storage.conservation, weekly_storage.flood, color='lightsteelblue', zorder=0)
    # ax.fill_between(weekly_storage.index, 0, weekly_storage.conservation, color='whitesmoke', zorder=0)
    # ax.set_title('{0} - {1}'.format(*attributes.loc[dam_id, ['GRAND_ID', 'DAM_NAME']]))
    # ax.text(0.01, 0.99, 'flood pool', va='top', transform=ax.transAxes)
    # ax.text(0.01, 0.01, 'conservation pool', va='bottom', transform=ax.transAxes)
    # ax.set(xlim=(weekly_storage.index.min(), weekly_storage.index.max()),
    #        ylim=(0, 1),
    #        ylabel='storage (% of capacity)');

  0%|          | 0/90 [00:00<?, ?it/s]

Fitting targets for dam 41: Ross


ValueError: "freq" must be either "W" for weekly or "D" for daily, not "flood"

### Release function

In [7]:
for dam_id in tqdm(reservoirs):
    
    # read reservoir attributes and extract storage capacity
    attributes = read_reservoir_attributes(GRanD_path, dam_id)
    Vtot = attributes.loc[dam_id, 'CAP_MCM']
    title = '{0} - {1}'.format(*attributes.loc[dam_id, ['GRAND_ID', 'DAM_NAME']])
    
    # fit release model
    model_release = fit_release(dam_id, USRDATS_path, GRanD_path, NOR_path=path_nor, cutoff_year=cutoff_year)

    # export fitted parameters
    pars_to_export = ['mean inflow (MCM/wk)', 'harmonic parameters', 'residual parameters', 'constraints']
    pars = {key: value for key, value in model_release.items() if key in pars_to_export}
    with open(path_release / f'{dam_id}.pkl', 'wb') as file:
             pickle.dump(pars, file)

    # extract info from the fitted release: average inflow, harmonic release (standardised) and release contraints
    avg_inflow = model_release['mean inflow (MCM/wk)']
    release_harmonic = create_release_harmonic(model_release['harmonic parameters']).set_index('epiweek').squeeze()
    release_linear = create_release_linear(model_release['residual parameters'])
    Qmin, Qmax = model_release['constraints']

    # combine weekly observed and harmonic releases
    weekly_release = (
        model_release['weekly release']
        # .merge(avg_inflow * (release_harmonic + 1), on='epiweek')
        .merge(release_harmonic, on='epiweek')
        .rename(columns={'release_harmonic': 'harmonic'})
        .sort_values(['year', 'epiweek'])
    )
    # weekly_release.sort_values(['year', 'epiweek'], inplace=True)
    weekly_release['date'] = weekly_release.apply(lambda row: epiweek_to_date(row['year'], row['epiweek']), axis=1)
    weekly_release.set_index('epiweek', inplace=True)
    
    plot_release(weekly_release.r, avg_inflow, release_harmonic, release_linear, Qmin, Qmax, title=title,
                 save=path_release / f'{dam_id}.jpg')

  0%|          | 0/90 [00:00<?, ?it/s]

Fitting release function for dam 41: Ross
Release residual model will be discarded; (release will be based harmonic function only)
Fitting release function for dam 63: Tieton
Release residual model will be discarded; (release will be based harmonic function only)
Fitting release function for dam 293: Fresno
Release residual model will be discarded; (release will be based harmonic function only)
Fitting release function for dam 300: Tiber Dike
Fitting release function for dam 307: Fort Peck Dam
Release residual model will be discarded; (release will be based harmonic function only)
Fitting release function for dam 319: Gibson
Fitting release function for dam 355: Yellowtail
Fitting release function for dam 362: Clark Canyon
Fitting release function for dam 364: Hebgen Dam
Fitting release function for dam 367: Mason
Release residual model will be discarded; (release will be based harmonic function only)
Fitting release function for dam 368: Lima
Release residual model will be discarded; 

In [ ]:
plt.hlines([None, None], 1, 52, color='k', ls=':', lw=.5, label='min-max')
plt.legend()